# Predicting Heart Disease Diagnosis with Serum Cholesterol and Blood Pressure

## Introduction

Heart disease is a major worldwide health concern because it is a common and potentially fatal disorder. To better understand, diagnose, and forecast heart illnesses, academics and healthcare practitioners are utilizing data-driven approaches thanks to technological advancements and the growing availability of health-related statistics. The UCI heart disease dataset stands as a valuable resource in this pursuit, offering a comprehensive set of attributes that can be analyzed to uncover patterns and insights related to cardiovascular health. The primary objective of this project is to address the question: **"Can we predict heart disease diagnoses based on serum cholesterol and blood pressure?"** using a knn classification model. High Serum cholesterol and blood pressure are two commonly cited risk factors for heart disease; high cholesterol levels can build up in the walls of the arteries which can decrease the blood flow to the heart and high blood pressure can cause arteries to become hard and thick which can also decrease blood flow to the heart.

The data set includes many observations of individuals admitted to  a Cleveland hospital with 14 variables, the two relevant to our preliminary analysis are:
1) chol: serum cholestoral	(mg/dl)
2) trest_bps: resting blood pressure (on admission to the hospital)	(mm Hg)

## Methods & Results

## Discussion

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quisque gravida porta nibh, at convallis nisl sodales gravida. Curabitur non tincidunt orci. Praesent malesuada dapibus lectus at maximus. Praesent hendrerit libero sed ligula aliquam vehicula. Suspendisse nec ultricies massa, vel vulputate nunc. Quisque lacus leo, rutrum vitae arcu ut, molestie hendrerit orci. Ut convallis ac metus vel lacinia. Ut nec faucibus eros, ut dictum augue. Morbi posuere sem et congue sagittis. Curabitur vitae eros eget nulla interdum luctus. Nunc condimentum nibh in mauris accumsan mattis. Quisque faucibus, nisi quis faucibus pellentesque, sapien purus laoreet tortor, quis venenatis leo tellus sit amet velit.

## References

Janosi,Andras, Steinbrunn,William, Pfisterer,Matthias, and Detrano,Robert. (1988). Heart Disease. UCI Machine Learning Repository. https://doi.org/10.24432/C52P4X.